In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    '../../datasets/lending_club/lending_club_2015.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/home/conner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,12000.0,12000.0,12000.0,60 months,9.80%,253.79,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,25000.0,25000.0,25000.0,36 months,5.32%,752.87,A,A1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,28000.0,28000.0,28000.0,60 months,18.49%,718.51,D,D5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,30000.0,30000.0,30000.0,36 months,15.77%,1051.31,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,17600.0,17600.0,17600.0,60 months,19.89%,465.22,E,E3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [4]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
'''
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)
''';

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [5]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
2
term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
39
next_pymnt_d
4
last_credit_pull_d
41
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
14
hardship_end_date
15
payment_plan_start_date
15
hardship_loan_status
4
disbursement_method
1
debt_settlement_flag
2
debt_settlement_flag_date
32
settlement_status
3
settlement_date
35


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [6]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [7]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421092,NaN,NaN,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421093,NaN,NaN,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421094,NaN,NaN,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [9]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_May-2017,settlement_date_Nov-2015,settlement_date_Nov-2016,settlement_date_Nov-2017,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017
0,NaN,NaN,12000.0,12000.0,12000.0,9.80,253.79,65000.0,23.84,0.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,25000.0,25000.0,25000.0,5.32,752.87,150000.0,9.54,0.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,28000.0,28000.0,28000.0,18.49,718.51,75000.0,31.88,0.0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,30000.0,30000.0,30000.0,15.77,1051.31,175000.0,18.50,0.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,17600.0,17600.0,17600.0,19.89,465.22,44000.0,17.56,0.0,...,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,33600.0,33600.0,33600.0,10.78,1096.53,104000.0,14.01,0.0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,NaN,11550.0,11550.0,11550.0,16.59,284.51,38000.0,21.07,0.0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,20000.0,20000.0,20000.0,13.99,465.27,70000.0,16.90,0.0,...,0,0,0,0,0,0,0,0,0,1
9,NaN,NaN,12000.0,12000.0,12000.0,10.78,391.62,40000.0,31.95,3.0,...,0,0,0,0,0,0,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/home/conner/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.98967065, 0.98962316, 0.9887443 , 0.98819786, 0.98743766,
       0.98738957, 0.98689085, 0.98489562, 0.98560809, 0.89065691])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [47]:
# pca on X
# normalize data
from sklearn import preprocessing
from sklearn.decomposition import PCA

X_scaled = pd.DataFrame(preprocessing.scale(X),columns = X.columns) 

# PCA
components = 10
pca = PCA(n_components=components)
pca.fit_transform(X_scaled);

In [78]:
# collect unique top 10 of abs(components_) from each PCA
pca_components = pd.DataFrame(pca.components_,columns=X_scaled.columns,index = range(components))
pca_components = np.abs(pca_components)
top_components = set()
for index, row in pca_components.iterrows():
    top_row = row.sort_values(ascending=False)[:10]
    top_components.update(list(top_row.index))
X_components = X[list(top_components)]
print ("Keeping {} features".format(len(top_components)))
print (sorted(top_components))

Keeping 74 features
['acc_open_past_24mths', 'avg_cur_bal', 'collection_recovery_fee', 'debt_settlement_flag_N', 'debt_settlement_flag_Y', 'funded_amnt', 'funded_amnt_inv', 'grade_A', 'hardship_end_date_Apr-2018', 'hardship_end_date_Aug-2017', 'hardship_end_date_Dec-2017', 'hardship_flag_N', 'hardship_flag_Y', 'hardship_loan_status_Current', 'hardship_loan_status_Late (16-30 days)', 'hardship_reason_NATURAL_DISASTER', 'hardship_start_date_Jun-2017', 'hardship_start_date_Mar-2018', 'hardship_start_date_May-2017', 'hardship_start_date_Sep-2017', 'hardship_status_ACTIVE', 'hardship_status_COMPLETED', 'hardship_type_INTEREST ONLY-3 MONTHS DEFERRAL', 'home_ownership_MORTGAGE', 'home_ownership_RENT', 'installment', 'int_rate', 'last_credit_pull_d_Mar-2018', 'last_pymnt_amnt', 'last_pymnt_d_Mar-2018', 'loan_amnt', 'mort_acc', 'next_pymnt_d_Apr-2018', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_op_pas

In [76]:
# evaluate PCA'd featuress
rfc = ensemble.RandomForestClassifier()
Y = y2015['loan_status']

cross_val_score(rfc, X_components, Y, cv=10)

/home/conner/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.98895828, 0.98883955, 0.98943294, 0.98874403, 0.98855379,
       0.98836326, 0.98779329, 0.98679554, 0.98627307, 0.98356529])